In [3]:
# libraries
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [4]:
#read data in csv format
git = pd.read_csv('gitData.csv')
issues = pd.read_csv('gitIssues.csv')

In [ ]:
# check the shape of the data
print(git.shape)
df_issues.shape

In [ ]:
#data inspection
print(git.info())
df_issues.info()

In [5]:
##### gitissues
# let's connect to the localhost
client = MongoClient()

# let's create a database 
db = client.github

# collection for git issues
gitIssues = db.gitIssues

# print connection
print("""
Database
==========
{}

Collection
==========
{}
""".format(db, gitIssues), flush=True
)


Database
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github')

Collection
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github'), 'gitIssues')



In [7]:
# import the gitIssues csv to mongo compass
##
# slow loading of data
d_gitissue = {}
# pass data 
for i in issues.index:
    d_gitissue = {
        "Questions": {
            "Title": issues.loc[i,"title"],
            "Body": issues.loc[i,"body"]
        },
        "User" : {
            "Username" : issues.loc[i, "user"],
            "User_ID" :  issues.loc[i,"user_id"]
        },
        "State" : {
            "State" : issues.loc[i, "state"],
            "Created_at" :  issues.loc[i,"created_at"],
            "Updated_at" :  issues.loc[i,"updated_at"],
            "Closed_at" : issues.loc[i, "closed_at"]
        },
        "Assignees" : issues.loc[i, "assignees"],
        "Closed_by" : issues.loc[i, "closed_by"],
        "Labels" : issues.loc[i, "labels"],
        "Reactions" : issues.loc[i, "reactions"],
        "N_comments" : issues.loc[i, "n_comments"].astype(str),
        "Projects" : issues.loc[i, "project"]
    }
    break
    #gitIssues.insert_one(d)

In [8]:
###### gitdata
# new collection for git data
gitData = db.gitData

# print connection
print("""
Database
==========
{}

Collection
==========
{}
""".format(db, gitData), flush=True
)


Database
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github')

Collection
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github'), 'gitData')



In [9]:
# import the git data to mongo compass
#%%time 
##
# slow loading of data
d_gitdata = {}
#pass data
for i in git.index:
    d_gitdata = {
        "Commit": {
            "hash":git.loc[i, "hash"],
            "Msg":{
                "msg":git.loc[i, 'msg'],
                "Author":{
                    "Author_name": git.loc[i, 'author_name'],
                    "Author_date": git.loc[i, 'author_date'],
                    "Author_timezone": git.loc[i, 'author_timezone'].astype(str)
                }
            },
            "Committer":{
                "Committer_name": git.loc[i, 'committer_name'],
                "Committer_date": git.loc[i, 'committer_date'],
                "Committer_timezone": git.loc[i, 'committer_timezone'].astype(str)
            }
        },
        "Branch":{
            "Branches": git.loc[i, 'branches'],
            "In_main_branch": git.loc[i, 'in_main_branch'].astype(str),
            "Merge": git.loc[i, 'merge'].astype(str),
            "Parents":git.loc[i, 'parents']
        },
        "Project_name": git.loc[i, 'project_name'],
        "File":{
            "Filename": git.loc[i, 'filename'],
            "Change_type": git.loc[i, 'change_type'],
            "Commit_change":{
                "Deletions": git.loc[i, 'deletions'].astype(str),
                "Insertions": git.loc[i, 'insertions'].astype(str),
                "Files": git.loc[i, 'files'].astype(str),
                "Lines":git.loc[i, 'lines'].astype(str)
            }
        },
        "Code_change":{
            "Path":{
                "old_path":git.loc[i, 'old_path'],
                "new_path": git.loc[i, 'new_path']
            },
            "Diff":{
                "Diff": git.loc[i, 'diff'],
                "Diff_parse":{
                    "Diff_parsed":git.loc[i, 'diff_parsed'],
                    "Deleted_lines":git.loc[i, 'deleted_lines'].astype(str)
                }
            },
            "Source_code":{
                "Source_code":git.loc[i, 'source_code'],
                "Source_code_before":git.loc[i, 'source_code_before']
            }
        },
        "Nloc":git.loc[i, 'nloc'],
        "Complexity":git.loc[i, 'complexity'],
        "Token_count":git.loc[i, 'token_count']
        
    }
    break 
    #gitData.insert_one(d_gitdata)

In [10]:
##### issue comment
# new collection for issue comment
IssueComment = db.IssueComment

# print connection
print("""
Database
==========
{}

Collection
==========
{}
""".format(db, IssueComment), flush=True)



Database
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github')

Collection
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'github'), 'IssueComment')



In [11]:
d_gitcomment = {}
# pass data 
for i in issues.index:
    d_gitcomment = {
        "Title": issues.loc[i,"title"],
        "Comment_created_at": issues.loc[i,"comment_created_at"],
        "Labels" : issues.loc[i, "labels"],
        "Reactions" : issues.loc[i, "reactions"],
        "N_comments" : issues.loc[i, "n_comments"].astype(str),
        "Projects" : issues.loc[i, "project"]
    }
    break
    #IssueComment.insert_one(d_gitcomment)

# Data cleaning 

## Remove NA values

In [57]:
# our database
db = client.github

# collection for git issues
gitIssues = db.gitIssues
gitIssues_new = db.gitIssues_new
gitData = db.gitData
gitData_new = db.gitData_new
IssueComment = db.IssueComment
IssueComment_new = db.IssueComment_new

In [22]:
##### gitissues
# get key names for gitIssues collection
key_list_gitissue = []
for i in d_gitissue.keys():
    try:
        for b in d_gitissue.get(str(i)).keys():
            key_list_gitissue.append(str(i) + '.' + str(b))
    except:
        key_list_gitissue.append(i)

In [25]:
# unset NaN fields
for i in key_list_gitissue:
    update = gitIssues.update_many({str(i):np.nan},{"$unset": {str(i):""}})
    print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format(i, update.matched_count, update.modified_count), flush=True)


    Key: Questions.Title
    Matched: 0
    Modified: 0
    ------------
    

    Key: Questions.Body
    Matched: 461
    Modified: 461
    ------------
    

    Key: User.Username
    Matched: 0
    Modified: 0
    ------------
    

    Key: User.User_ID
    Matched: 0
    Modified: 0
    ------------
    

    Key: State.State
    Matched: 0
    Modified: 0
    ------------
    

    Key: State.Created_at
    Matched: 0
    Modified: 0
    ------------
    

    Key: State.Updated_at
    Matched: 0
    Modified: 0
    ------------
    

    Key: State.Closed_at
    Matched: 0
    Modified: 0
    ------------
    

    Key: Assignees
    Matched: 0
    Modified: 0
    ------------
    

    Key: Closed_by
    Matched: 19088
    Modified: 19088
    ------------
    

    Key: Labels
    Matched: 0
    Modified: 0
    ------------
    

    Key: Reactions
    Matched: 0
    Modified: 0
    ------------
    

    Key: N_comments
    Matched: 0
    Modified: 0
    ------------
    



In [27]:
##### gitData
# get key names for gitIssues collection
key_list_gitData = []
for i in d_gitdata.keys():
    try:
        for b in d_gitdata.get(str(i)).keys():
            key_list_gitData.append(str(i) + '.' + str(b))
    except:
        key_list_gitData.append(i)

In [28]:
# unset NaN fields
for i in key_list_gitData:
    update = gitData.update_many({str(i):np.nan},{"$unset": {str(i):""}})
    print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format(i, update.matched_count, update.modified_count), flush=True)


    Key: Commit.hash
    Matched: 0
    Modified: 0
    ------------
    

    Key: Commit.Msg
    Matched: 0
    Modified: 0
    ------------
    

    Key: Commit.Committer
    Matched: 0
    Modified: 0
    ------------
    

    Key: Branch.Branches
    Matched: 0
    Modified: 0
    ------------
    

    Key: Branch.In_main_branch
    Matched: 0
    Modified: 0
    ------------
    

    Key: Branch.Merge
    Matched: 0
    Modified: 0
    ------------
    

    Key: Branch.Parents
    Matched: 0
    Modified: 0
    ------------
    

    Key: Project_name
    Matched: 0
    Modified: 0
    ------------
    

    Key: File.Filename
    Matched: 0
    Modified: 0
    ------------
    

    Key: File.Change_type
    Matched: 0
    Modified: 0
    ------------
    

    Key: File.Commit_change
    Matched: 0
    Modified: 0
    ------------
    

    Key: Code_change.Path
    Matched: 0
    Modified: 0
    ------------
    

    Key: Code_change.Diff
    Matched: 0
    Modified: 0


In [31]:
#### IssueComment
# get key names for gitIssues collection
key_list_issuecomment = []
for i in d_gitcomment.keys():
    try:
        for b in d_gitcomment.get(str(i)).keys():
            key_list_issuecomment.append(str(i) + '.' + str(b))
    except:
        key_list_issuecomment.append(i)

In [32]:
# unset NaN fields
for i in key_list_issuecomment:
    update = IssueComment.update_many({str(i):np.nan},{"$unset": {str(i):""}})
    print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format(i, update.matched_count, update.modified_count), flush=True)


    Key: Title
    Matched: 0
    Modified: 0
    ------------
    

    Key: Comment_created_at
    Matched: 0
    Modified: 0
    ------------
    

    Key: Labels
    Matched: 0
    Modified: 0
    ------------
    

    Key: Reactions
    Matched: 0
    Modified: 0
    ------------
    

    Key: N_comments
    Matched: 0
    Modified: 0
    ------------
    

    Key: Projects
    Matched: 0
    Modified: 0
    ------------
    


## change datatype 

In [ ]:
# change the User_ID into integer

### change into integer

In [58]:
# change the data type of "N_comments" in gitIssues into integer
update = gitIssues_new.update_many(
    {},
    [{"$set": {
        "N_comments": {
            "$toInt":"$N_comments" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("N_comments", update.matched_count, update.modified_count), flush=True)



    Key: N_comments
    Matched: 32329
    Modified: 32329
    ------------
    


In [61]:
# change the data type of "Author_timezone" in gitData into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "Commit.Msg.Author.Author_timezone": {
            "$toInt":"$Commit.Msg.Author.Author_timezone" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Author_timezone", update.matched_count, update.modified_count), flush=True)



    Key: Author_timezone
    Matched: 60996
    Modified: 60996
    ------------
    


In [62]:
# change the data type of "N_comments" in IssueComment into integer
update = IssueComment_new.update_many(
    {},
    [{"$set": {
        "N_comments": {
            "$toInt":"$N_comments" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("N_comments", update.matched_count, update.modified_count), flush=True)


    Key: N_comments
    Matched: 32329
    Modified: 32329
    ------------
    


In [63]:
# change the data type of "Deletions" in gitdata into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "File.Commit_change.Deletions": {
            "$toInt":"$File.Commit_change.Deletions" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Deletions", update.matched_count, update.modified_count), flush=True)


    Key: Deletions
    Matched: 60996
    Modified: 60996
    ------------
    


In [69]:
# change the data type of "Insertions" in gitdata into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "File.Commit_change.Insertions": {
            "$toInt":"$File.Commit_change.Insertions" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Insertions", update.matched_count, update.modified_count), flush=True)


    Key: Insertions
    Matched: 60996
    Modified: 60996
    ------------
    


In [ ]:
# change the data type of "Files" in gitdata into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "File.Commit_change.Files": {
            "$toInt":"$File.Commit_change.Files" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Files", update.matched_count, update.modified_count), flush=True)

In [ ]:
# change the data type of "Lines" in gitdata into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "File.Commit_change.Lines": {
            "$toInt":"$File.Commit_change.Lines" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Lines", update.matched_count, update.modified_count), flush=True)

In [ ]:
# change the data type of "Deleted Lines" in gitdata into integer
update = gitData_new.update_many(
    {},
    [{"$set": {
        "Code_change.Diff.Diff_parse.Deleted_lines": {
            "$toInt":"$Code_change.Diff.Diff_parse.Deleted_lines" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Deleted Lines", update.matched_count, update.modified_count), flush=True)

### change datatype to datetime

In [66]:
# change the data type of 'Created_at', 'Updated_at', and 'closed_at' in gitissues
update = gitIssues_new.update_many(
    {},
    [{"$set": {
        "State.Created_at": {
            "$toDate":"$State.Created_at" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Created_at", update.matched_count, update.modified_count), flush=True)


    Key: Created_at
    Matched: 32329
    Modified: 32329
    ------------
    


In [ ]:
# change the data type of 'Created_at', 'Updated_at', and 'closed_at' in gitissues
update = gitIssues_new.update_many(
    {},
    [{"$set": {
        "State.Closed_at": {
            "$toDate":"$State.Closed_at" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Closed_at", update.matched_count, update.modified_count), flush=True)

In [ ]:
# change the data type of 'Created_at', 'Updated_at', and 'closed_at' in gitissues
update = gitIssues_new.update_many(
    {},
    [{"$set": {
        "State.Updated_at": {
            "$toDate":"$State.Updated_at" } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Updated_at", update.matched_count, update.modified_count), flush=True)

In [ ]:
# change the data type of 'Created_at', 'Updated_at', and 'closed_at' in gitissues
update = gitData_new.update_many(
    {},
    [{"$set": {
        "Commit.Msg.Author.Author_date": {
            "$toDate":"$Commit.Msg.Author.Author_date } 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Author_date", update.matched_count, update.modified_count), flush=True)

In [67]:
##### don't run this 
# convert 'True' or False into boolean
update = gitData_new.update_many(
    {},
    [{"$set": {
        "Branch.In_main_branch": {
            "$toBool":'$Branch.In_main_branch'} 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("In_main_branch", update.matched_count, update.modified_count), flush=True)


    Key: In_main_branch
    Matched: 60996
    Modified: 60996
    ------------
    


In [68]:
###### don't run this
# convert 'True' or False into boolean
update = gitData_new.update_many(
    {},
    [{"$set": {
        "Branch.Merge": {
            "$toBool":'$Branch.Merge'} 
        }
    }
]
)

print("""
    Key: {}
    Matched: {}
    Modified: {}
    ------------
    """.format("Merge", update.matched_count, update.modified_count), flush=True)


    Key: Merge
    Matched: 60996
    Modified: 60996
    ------------
    


# descriptive analytics

In [75]:
# check for number of comments in pytorch
db.IssueComment.find(
    {
        "Projects": {
            $in: ["pytorch"]
        }
    }.itcount()
); 

# Pyspark